# Import

In [1]:
import numpy as np
from fatiando import mesher, gridder, utils
from scipy.linalg import toeplitz
from sys import getsizeof
from timeit import default_timer as time
%matplotlib inline

# Observation and Equivalent layer grids

In [2]:
# Create a regular grid at 0m height
#area = [1, 4, 1, 3]
area = [0, 100, 0, 50]
shape = (50, 50)
xi, yi, zi = gridder.regular(area, shape, z=0.)

# Equivalent Layer
xj, yj, zj = gridder.regular(area, shape, z=80)

# Create the true parameters vector
N = shape[0]*shape[1]
p_true = np.arange(0,N,1)
p_true2 = p_true.reshape(N,1)

# Classic Forward Problem Construction - MAG

In [4]:
# Magnetic Configuration
inc0 = np.deg2rad(-30.)
dec0 = np.deg2rad(40.)
inc = np.deg2rad(15.)
dec = np.deg2rad(59.)
F = np.array([np.cos(inc0)*np.cos(dec0), np.cos(inc0)*np.sin(dec0), np.sin(inc0)])
h = np.array([np.cos(inc)*np.cos(dec), np.cos(inc)*np.sin(dec), np.sin(inc)])

In [5]:
# Calculo da matriz de sensibilidade
A = np.zeros((N, N), dtype=np.float)

s = time()
for i in xrange (N):
    a = (xj-xi[i])
    b = (yj-yi[i])
    c = (zj-zi[i])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5
    A[i] = ((F[0]*Hxx+F[1]*Hxy+F[2]*Hxz)*h[0] + (F[0]*Hxy+F[1]*Hyy+F[2]*Hyz)*h[1] + (F[0]*Hxz+F[1]*Hyz+F[2]*Hzz)*h[2])

dobs_classic = A.dot(p_true2)

e = time()
tcpu = e - s
print tcpu
print A.nbytes/(1024.*1024.)

0.9546041854
47.6837158203


# First Row to Toeplitz Blocks - Forward Model

In [10]:
s = time()

dobs_bt = np.zeros(N).reshape(N,1)

l = shape[1]

for i in range (shape[1]):
    k = 0 # index of the parameter's segment
    j = i # index of the data's segment using the lower matrix
    
    a = (xj[shape[0]*(i):shape[0]*(i+1)]-xi[0])
    b = (yj[shape[0]*(i):shape[0]*(i+1)]-yi[0])
    c = (zj[shape[0]*(i):shape[0]*(i+1)]-zi[0])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5
    W_bt_0 = ((F[0]*Hxx+F[1]*Hxy+F[2]*Hxz)*h[0] + (F[0]*Hxy+F[1]*Hyy+F[2]*Hyz)*h[1] + (F[0]*Hxz+F[1]*Hyz+F[2]*Hzz)*h[2])
    
    a = (xj[shape[0]*(i):shape[0]*(i+1)]-xi[shape[0]-1])
    b = (yj[shape[0]*(i):shape[0]*(i+1)]-yi[shape[0]-1])
    c = (zj[shape[0]*(i):shape[0]*(i+1)]-zi[shape[0]-1])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5
    W_bt_1 = ((F[0]*Hxx+F[1]*Hxy+F[2]*Hxz)*h[0] + (F[0]*Hxy+F[1]*Hyy+F[2]*Hyz)*h[1] + (F[0]*Hxz+F[1]*Hyz+F[2]*Hzz)*h[2])
    
    a = (xj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-xi[-shape[0]])
    b = (yj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-yi[-shape[0]])
    c = (zj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-zi[-shape[0]])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5
    W_bt_2 = ((F[0]*Hxx+F[1]*Hxy+F[2]*Hxz)*h[0] + (F[0]*Hxy+F[1]*Hyy+F[2]*Hyz)*h[1] + (F[0]*Hxz+F[1]*Hyz+F[2]*Hzz)*h[2])

    a = (xj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-xi[-1])
    b = (yj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-yi[-1])
    c = (zj[shape[0]*(shape[0]-i-1):shape[0]*(shape[0]-i)]-zi[-1])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5
    W_bt_3 = ((F[0]*Hxx+F[1]*Hxy+F[2]*Hxz)*h[0] + (F[0]*Hxy+F[1]*Hyy+F[2]*Hyz)*h[1] + (F[0]*Hxz+F[1]*Hyz+F[2]*Hzz)*h[2])

    block_u = toeplitz(W_bt_1[::-1], W_bt_0) # create each Toeplitz block by the segment of the first row
    block_l = toeplitz(W_bt_3[::-1], W_bt_2) # create each Toeplitz block by the segment of the last row

    while k < l:
        
        # lower matrix
        block_p = block_l.dot(p_true2[shape[0]*(k):shape[0]*(k+1)])
        dobs_bt[shape[0]*(j):shape[0]*(j+1)] += block_p
        if i > 0:
        # upper matrix
            block_p = block_u.dot(p_true2[shape[0]*(j):shape[0]*(j+1)])
            dobs_bt[shape[0]*(k):shape[0]*(k+1)] += block_p
        
        k += 1
        j += 1
    l -= 1

e = time()
tcpu = e - s
print tcpu
print block.nbytes/(1024.*1024.)

0.0203975679725
0.0190734863281


In [11]:
np.allclose(dobs_classic, dobs_bt)

True